## Part II
# Binomial Tree: option valuation (50 pts.)
 
 A commonly used approach to compute the price of an option is the so-called binomial tree method. Suppose that the maturity of an option on a non-dividend-paying stock St is divided into N subintervals of length ∆t. We will refer to the j th node at time i · ∆t as the (i, j) node. The stock price at the (i, j) node is Si,j = S0u^j * d^(i-j) (with u and d the upward and downward stock price movements, respectively). In the binomial tree approach, option prices are computed through a backward induction scheme:

1. The value of a call option at its expiration date is max{$0, S_{N,j} − K$};
2. Suppose that the values of the option at time (i + 1) · ∆t is known for all j. There is a risk-neutral probability p of moving from the (i, j) node at time i · ∆t to the (i + 1, j + 1) node at time (i + 1) · ∆t , and a probability 1 − p of moving from the (i, j) node at time i · ∆t to the (i + 1, j) node at time (i + 1) · ∆t . Risk-neutral valuation gives $f_{i,j} = e^{-r\Delta t} \left( p f_{i+1,j+1} + (1 - p) f_{i+1,j} \right)$

A general introduction to the binomial tree model and a detailed derivation of the relevant parameters is provided in appendix A. Please read the appendix carefully before starting on the assignment.

Consider a European call option on a non-dividend-paying stock with a maturity of one year and strike price of €99. Let the one year interest rate be 6% and the current price of the stock be €100. Furthermore, assume that the volatility is 20%.

In [ ]:
import numpy as np

def buildTree(S, vol, T, N):
    dt = T / N

    matrix = np.zeros((N + 1, N + 1))

    u = 0 # TODO
    d = 0 # TODO
    
    for i in np.arange(N + 1):
        for j in np.arange(i + 1):
            # Hint: express each cell as a combination of up and down moves
            matrix[i, j] = 0 # TODO
            
    return matrix

In [ ]:
sigma = 0.1
S = 80
T = 1
N = 2
buildTree(S, sigma, T, N)

In [ ]:
def valueOptionMatrix(tree, T, r, K, vol):

    dt = T / N

    u = 0 # TODO
    d = 0 # TODO

    p = 0 # TODO

    columns = tree.shape[1]
    rows = tree.shape[0]
    
    # Walk backward, we start in last row of the matrix

    # Add the payoff function in the last row
    for c in np.arange(columns):
        S = tree[rows - 1, c] # value in the matrix
        tree[rows - 1, c] = 0 # TODO
    
    # For all other rows, we need to combine from previous rows
    # We walk backwards, from the last row to the first row
    for i in np.arange(rows - 1)[::-1]:
        for j in np.arange(i + 1):
            down = tree[i + 1, j]
            up = tree[i + 1, j + 1]
            tree[i, j] = 0 # TODO
            
    return tree

In [ ]:
sigma = 0.1
S = 80
T = 1
N = 2

K = 85
r = 0.1

tree = buildTree(S, sigma, T, N)
valueOptionMatrix(tree, T, r, K, sigma)

In [ ]:
# Play around with different ranges of N and step sizes.
N = np.arange(1, 300)

# Calculate the option price for the correct parameters
optionPriceAnalytical = 0 # TODO

# Calculate option price for each n in N
for n in N:
    treeN = buildTree(...) # TODO
    priceApproximatedly = valueOption(...) # TODO

# use matplotlib to plot the analytical value
# and the approximated value for each n